In [31]:
! pip install chembl_webresource_client

In [32]:
import csv
from chembl_webresource_client.new_client import new_client

In [33]:
# Mapping resulting structure with ChEMBL IDs into target uniprot IDs
compounds2targets = dict()

In [34]:
# Parse the CSV file to extract compounds ChEMBL IDs:
with open('tryptase_bioactivity_preprocessed_data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        compounds2targets[row[0]] = set()

In [35]:
# Process in chunks since we have 192 rows of ChEMBL IDs
chunk_size = 50
keys = list(compounds2targets.keys())

for i in range(0, len(keys), chunk_size):
  # Jumping from compounds to targets through activities
  activities = new_client.activity.filter(molecule_chembl_id__in=keys[i:i + chunk_size]).only(
      ['molecule_chembl_id', 'target_chembl_id'])
  for act in activities:
    compounds2targets[act['molecule_chembl_id']].add(act['target_chembl_id'])

In [36]:
# Now that our dictionary maps from compound ChEMBL IDs have been changed into target ChEMBL IDs
# We will replace target ChEMBL IDs with uniprot IDs

for key, val in compounds2targets.items():
    # Process in chunks
    lval = list(val)
    genes = set()
    for i in range(0, len(val), chunk_size):
        targets = new_client.target.filter(target_chembl_id__in=lval[i:i + chunk_size]).only(
            ['target_components'])
        for target in targets:
          for component in target['target_components']:
            for synonym in component['target_component_synonyms']:
              if synonym['syn_type'] == 'GENE_SYMBOL':
                genes.add(synonym['component_synonym'])

    compounds2targets[key] = genes

In [37]:
# Write it to the output CSV file
with open('tryptase_compounds_2_genes.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key, val in compounds2targets.items():
        writer.writerow([key] + list(val))

After looking at the *tryptase_compounds_2_genes.csv*, we can see that a majority of the 3 main genes associated with Tryptase are TPSAB1 (Tryptase alpha/beta-1), TPSG1 (Tryptase gamma), and TPSD1 (Tryptase delta). After a quick google search, we can find that **Cetrizine** or **Omalizumab** are the only drugs out there that target TPSD1 and TPSAB1 genes for inhibition of Tryptase, respectively.

As many know, Cetrizine is an FDA drug used an anti-allergic agent. Omalizumab is an injection that blocks the symptoms of allergic asthma, nasal polyps, and hives. This makes sense because high levels of Tryptase are associated with the release of histamine molecules from the mast cells, which trigger an allergic reaction.

Now, let's see what genes are associated with PD-L1...

